In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
df = pd.read_csv("cc1/airline_satisfaction.csv")
df.drop('Unnamed: 0', inplace = True, axis=1)
df =df.dropna()

In [108]:
df = df.sample(n=50000)

In [2]:
ordinal = ["Departure/Arrival time convenient","Ease of Online booking","Gate location",
           "Food and drink","Online boarding","Seat comfort","Inflight entertainment","On-board service","Leg room service",
           "Baggage handling","Checkin service","Inflight service","Cleanliness"]
df["Note_totale"]=df["Inflight wifi service"]
print(len(ordinal))
for i in ordinal:
    df["Note_totale"]+=df[i]

13


In [3]:
ordinal += ["Inflight wifi service"]
target=["satisfaction"]
quantitative =["Age","Flight Distance","Arrival Delay in Minutes","Departure Delay in Minutes","Note_totale"]
ordinal+=["Class"]
qualitative=["Gender","Customer Type","Type of Travel"]
features=quantitative+qualitative+ordinal

Preprocessing : 


In [4]:
X_qualitative = df[qualitative].values
X_ordinal = df [ordinal].values
X_quantitative_n = df[["Age","Arrival Delay in Minutes","Departure Delay in Minutes","Note_totale"]].values
X_quantitative_2 = df[["Flight Distance"]].values

In [5]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
#one_hot_encoder.fit(X_qualitative)
one_hot_encoded_X = one_hot_encoder.fit_transform(X_qualitative)
one_hot_encoded_X.toarray()
one_hot_encoder.get_feature_names_out()
X_encoded = np.concatenate([one_hot_encoded_X.toarray(),X_qualitative], axis =1) # faire ça pour chaque puis insert

In [6]:
one_hot_encoded_X

<129487x6 sparse matrix of type '<class 'numpy.float64'>'
	with 388461 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_vars = []
for categorical_var in X_ordinal.T:
    encoded_vars.append(label_encoder.fit_transform(categorical_var))
    print(f"===== Encoded classes: {label_encoder.classes_}")

===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: [0 1 2 3 4 5]
===== Encoded classes: ['Business' 'Eco' 'Eco Plus']


In [8]:
X_ordinal = np.array(encoded_vars).T

In [9]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_min_max_scaled = min_max_scaler.fit_transform(X=X_quantitative_2)

In [10]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
standard_scaled = standard_scaler.fit_transform(X=X_quantitative_n)

In [30]:
X_encoded = pd.concat([pd.DataFrame(one_hot_encoded_X.toarray()),pd.DataFrame(X_ordinal),pd.DataFrame(X_min_max_scaled),pd.DataFrame(standard_scaled)],axis=1)

In [34]:
X_encoded.describe()

,0,1,2,3,4,5,0,1,2,3,...,10,11,12,13,14,0,0,1,2,3
count,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,...,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,1.294870e+05,1.294870e+05,1.294870e+05,1.294870e+05
mean,0.507410,0.492590,0.816862,0.183138,0.690764,0.309236,3.057349,2.756786,2.976909,3.204685,...,3.306239,3.642373,3.286222,2.728544,0.593704,0.234089,-5.549664e-16,1.934504e-15,1.292855e-15,-2.133626e-16
std,0.499947,0.499947,0.386781,0.386781,0.462180,0.462180,1.526787,1.401662,1.278506,1.329905,...,1.266146,1.176614,1.313624,1.329235,0.621371,0.201446,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.145109e+00,-3.923289e-01,-3.860357e-01,-3.278910e+00
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,...,3.000000,3.000000,2.000000,2.000000,0.000000,0.077342,-8.221419e-01,-3.923289e-01,-3.860357e-01,-6.883924e-01
50%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,4.000000,3.000000,3.000000,1.000000,0.164176,3.778650e-02,-3.923289e-01,-3.860357e-01,6.717516e-02
75%,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,5.000000,4.000000,4.000000,1.000000,0.345921,7.654182e-01,-5.436375e-02,-6.968615e-02,7.148045e-01
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,2.000000,1.000000,3.014462e+00,4.078743e+01,4.158301e+01,2.657693e+00


In [29]:
pd.DataFrame(one_hot_encoded_X.toarray()).head()
#one_hot_encoded_X.shape

,0,1,2,3,4,5
0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,1.0,0.0,1.0,0.0


In [21]:
X_ordinal.shape

(129487, 15)

In [22]:
X_min_max_scaled.shape

(129487, 1)

In [23]:
standard_scaled.shape

(129487, 4)